In [84]:
import pandas as pd
import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import xml.etree.ElementTree as et
import numpy as np

In [85]:
df = pd.read_csv('train/golden_truth.txt', sep=" ")
df

,id,label
0,subject671,1
1,subject9917,1
2,subject6238,1
3,subject8581,1
4,subject7238,1
...,...,...
335,subject2182,0
336,subject9829,0
337,subject3270,0
338,subject6464,0


In [86]:
def parse_XML(xml_file, df_cols): 
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    for node in xroot: 
        res = []
        res.append(node.attrib.get(df_cols[0]))
        for el in df_cols[1:]: 
            if node is not None and node.find(el) is not None:
                res.append(node.find(el).text)
            else: 
                res.append(None)
        rows.append({df_cols[i]: res[i] 
                     for i, _ in enumerate(df_cols)})
    
    out_df = pd.DataFrame(rows, columns=df_cols)
        
    return out_df

train = pd.DataFrame()

for index, row in df.iterrows():
    df = parse_XML('train/data/'+row['id']+'.xml', ['ID','DATE', 'TEXT'])
    df['truth'] = row['label']
    df['ID'] = row['id']
    train = pd.concat([train, df])
    


In [94]:
train.dropna(inplace=True, thresh=3)
train = train.replace(to_replace='None', value=np.nan).dropna()

In [95]:
def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

tokenizer = RegexpTokenizer(r'\w+')

def remove_stopwords(text):
    words = " ".join([w for w in text if w not in stopwords.words('english')])
    return words

train['TEXT'] = train['TEXT'].apply(lambda x: remove_punctuation(x), 1)
train['TEXT'] = train['TEXT'].apply(lambda x: tokenizer.tokenize(x.lower()))
train['TEXT'] = train['TEXT'].apply(lambda x: remove_stopwords(x))

train

,ID,DATE,TEXT,truth
1,subject671,2018-05-03 04:56:17,theres pizza place couple doors couple kids wo...,1
2,subject671,2018-05-03 05:06:23,pour hot water pitcher americano shots pouring...,1
3,subject671,2018-05-03 06:04:10,ombre pink drink cool lime base think lot cust...,1
4,subject671,2018-05-03 07:08:17,tall americano double sleeve cup,1
5,subject671,2018-05-10 04:16:03,print blank receipt paper write closing list c...,1
...,...,...,...,...
655,subject8721,2018-08-28 18:15:06,sad say 8217t experience,0
656,subject8721,2018-08-28 18:15:18,waiting,0
657,subject8721,2018-08-28 18:23:38,8217s said,0
658,subject8721,2018-08-29 06:50:51,solid,0


In [96]:
train.to_csv('train.csv', header = True, index=False)